In [1]:
#Omero
import ezomero
from omero.model import OriginalFileI, PlateI, ScreenPlateLinkI, ScreenI, WellI, WellSampleI, ImageI
from omero.rtypes import rint, rlong, rstring, robject, unwrap


import getpass

In [2]:
#Parameters

# OMERO IDs
results_screen_id =  852  # This is the id under which the plate will be posted, I gues in our case the "WS44 screen"
plate_id = 765 # This is the plate with the "original data"
dataset_id = 8026 # This is the dataset of the "results - images"

In [3]:
# Login to OMERO
OMEROUSER = input(f"Enter username: \t")
OMEROPASS = getpass.getpass(prompt = f"Enter password: \t")


OMEROHOST = 'omero-imaging.uni-muenster.de'
OMEROPORT = 4064
OMEROWEB = 'https://omero-imaging.uni-muenster.de/webclient/'

conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)


Enter username: 	sweische
Enter password: 	········


In [4]:
# Get dataset
dataset = conn.getObject("Dataset", dataset_id)

# !!! This one you might want to skip and link the plate to our WS44 Screen !!!
# Create new plate
#results_screen_name = "Results_of_IDR0028_Plate1A"
#results_screen_id = ezomero.post_screen(conn, results_screen_name, description="These are the results of the CP analysis.") 


# Create new plate
plate = conn.getObject("Plate", plate_id)
plate_name = plate.name + "_Results"
results_plate = PlateI()
results_plate.name = rstring(plate_name)
results_plate = conn.getUpdateService().saveAndReturnObject(results_plate)
results_plate_id = results_plate.getId()


# Links new Plate with new Screen
link = ScreenPlateLinkI()
link.setParent(ScreenI(results_screen_id, False))
link.setChild(PlateI(results_plate_id, False))
link_update_service = conn.getUpdateService()
link_update_service.saveObject(link)

In [5]:
rows_columns = []
image_ids = []

for image in dataset.listChildren():
    image_ids.append(image.id)
    rows_columns.append((int(image.name[:3]), int(image.name[3:6])))

In [6]:
#conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
for (row, column) in set(rows_columns):
    images_select = []
    print(f"Dataset to Plate: {row, column}")
    for i, rc in enumerate(rows_columns): 
        if rc == (row, column):
            images_select.append(image_ids[i])
    
    update_service = conn.getUpdateService()

    well = WellI()
    well.plate = PlateI(results_plate_id, False)
    well.column = rint(column)
    well.row = rint(row)
    
#try:
    for image in images_select:
        ws = WellSampleI()
        ws.image = ImageI(image, False)
        ws.well = well
        well.addWellSample(ws)
    update_service.saveObject(well)
#except Exception:
#print("error")   

Dataset to Plate: (3, 4)
Dataset to Plate: (7, 13)
Dataset to Plate: (3, 3)
Dataset to Plate: (7, 15)
Dataset to Plate: (3, 5)
Dataset to Plate: (7, 14)


In [7]:
# remove from Dataset

remove_from = conn.getObject("Dataset", dataset_id)
for image_id in image_ids:
    if remove_from is not None:
        image = conn.getObject("Image", image_id)
        links = list(image.getParentLinks(remove_from.id))
        link_ids = [l.id for l in links]
        conn.deleteObjects('DatasetImageLink', link_ids)


In [8]:
conn.close()